In [5]:
!pip install ipython-autotime
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 4.33 s (started: 2024-09-25 17:31:45 +00:00)


Nhược điểm chung của Random Search và Grid Search là đều hoạt động theo một cách tìm kiếm mà không biết trước thông tin cơ bản nào (uninformed). Giả sử một số lần test phát hiện ra một số hyperparameters nhất định không ảnh hưởng đến kết quả hoặc các khoảng giá trị nhất định không thực sự hiệu quả thì những thông tin này không được lan truyền tới các lần tìm kiếm sau. Vì lý do này, Halving Search ra đời với HalvingGridSearch và HalvingRandomSearch được sử dụng để tìm kiếm trên một parameter space bằng cách áp dụng cơ chế thực nghiệm với những lần chia đôi liên tiếp trên Grid Search và Random Search.

Sau khi giảm không gian xuống một nửa, một số lượng lớn các kết hợp hyperparameters được đánh giá trong một vòng thực nghiệm ban đầu nhưng sử dụng tài nguyên tính toán ít hơn. Điều này đạt được bằng cách chạy thử nghiệm trên một mẫu phụ của một vài trường hợp từ dataset. Vòng đầu tiên cho phép lựa chọn một tập hợp con của các hyperparameter, với những hyperparameter tốt nhất trên vòng 1 sẽ được sử dụng cho vòng thứ 2 với kích thước dataset được tăng lên. Và các vòng tiếp theo làm tương tự.

In [6]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

time: 622 µs (started: 2024-09-25 17:31:50 +00:00)


In [7]:
X, y = make_classification(n_samples=300, n_features=50,
                          n_informative=10,
                          n_redundant=25, n_repeated=15,
                          n_clusters_per_class=5,
                          flip_y=0.05, class_sep=0.5,
                          random_state=0)

time: 11.6 ms (started: 2024-09-25 17:31:54 +00:00)


In [9]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn import svm
from scipy.stats import loguniform
from sklearn import model_selection

svc = svm.SVC()
svc = svm.SVC(probability=True, random_state=1)

search_dict = {'kernel': ['linear', 'rbf'],
                'C': loguniform(1, 1000),
                'gamma': loguniform(0.0001, 0.1)
              }

scorer = 'accuracy'

time: 14.6 ms (started: 2024-09-25 17:33:01 +00:00)


In [10]:
search_func = HalvingRandomSearchCV(estimator=svc,
                                  param_distributions=search_dict,
                                  resource='n_samples',
                                  max_resources=100,
                                  aggressive_elimination=True,
                                  scoring=scorer,
                                  n_jobs=-1,
                                  cv=5,
                                  random_state=0)

time: 755 µs (started: 2024-09-25 17:33:08 +00:00)


In [ ]:
search_func.fit(X, y)
print (search_func.best_params_)
print (search_func.best_score_)

In [12]:
# Mo ta technical de ket hop GridSearch va Random Search
# Dataset chay cham => Giam data di 1 nua
# A giam di 1/2 => B
# B su dung Gridsearch
search_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
  'kernel': ['rbf']}
]
=> Optimize {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

# A su dung RandomSearch
search_grid = [
  {'kernel': ['rbf'], 'C': loguniform[1,13],'gamma': loguniform[0.0001, 0.01]},
]


time: 449 µs (started: 2024-09-25 17:34:42 +00:00)
